In [0]:
import os
import matplotlib
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import scipy.misc
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import zipfile
from sklearn import preprocessing
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Add, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Concatenate
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input


from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [21]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [22]:
with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/gdrive/My\ Drive/foo.txt

Hello Google Drive!

In [0]:

test = pd.read_csv("./gdrive/My Drive/mafat/test.csv")
test.set_index('image_id',inplace=True)


In [0]:
image_size=64
num_channels=3
num_pictures=275
train_dir="./gdrive/My Drive/mafat/training imagery/"
val_dir="./gdrive/My Drive/mafat/test imagery/"


In [6]:
image_name=[]
num_tiff=0
for file in os.listdir(train_dir):
  if file[-3:]!='jpg':
    num_tiff=num_tiff+1
    continue
  image_name.append(file[:-4])
print(num_tiff)

80


In [7]:
total_pictures=len(image_name)
total_pictures

1583

In [0]:
mymodel=load_model("./gdrive/My Drive/mafat/color/MyModel_color")

In [9]:
val_image_name=[]
tiff=0
tif=0
jpg=0
for file in os.listdir(val_dir):

  if file[-3:]=='tif':
    tif+=1
    val_image_name.append(file)
  if file[-3:]=='iff':
    tiff+=1
    val_image_name.append(file)
    
  if file[-3:]=='jpg':
    jpg+=1
    val_image_name.append(file)
    
print('tif=',tif)
print('tiff=',tiff)
print('jpg=',jpg)

tif= 9
tiff= 2
jpg= 2531


In [0]:
#importing data


def get_next_val_batch(image_index,num_pictures,last_index):
  
  images=np.ndarray(shape=(num_pictures,image_size,image_size,num_channels))
  current_index=0
  tag_id=[]
  
   

  while current_index<num_pictures and image_index<len(val_image_name):
    
      file=val_image_name[image_index]
      image=os.path.join(val_dir,file)
      
      file_data=mpimg.imread(image)
      height=file_data.shape[0]
      breadth=file_data.shape[1]
    
      image_id=np.ndarray(shape=(1),dtype='int32')
      if file[-3:]=='iff':
        file=file[:-5]
      else:
        file=file[:-4]
      
      image_id=int(file)
    
    
      lb=test.loc[image_id,['tag_id']]
      label_gate=0
      if(lb.shape[0]==1):
          lb=lb.values
          label_gate=1
      else:
          lb=lb['tag_id'].tolist()
    
    
      bb=test.loc[image_id,['p1_x','p_1y',' p2_x',' p2_y',' p3_x',' p3_y',' p4_x',' p4_y']].values
      n=bb.shape[0]
      if len(bb.shape)==1:
        box=np.ndarray(shape=(1,8))
        box[0,:]=bb[0]
        bb=box
        n=1
    
      box=np.ndarray(shape=(n,4))
      for i in range(n):
        x1=min(bb[i,0],bb[i,2],bb[i,4],bb[i,6])
        x2=max(bb[i,0],bb[i,2],bb[i,4],bb[i,6])
        y1=min(bb[i,1],bb[i,3],bb[i,5],bb[i,7])
        y2=max(bb[i,1],bb[i,3],bb[i,5],bb[i,7])
        box[i,0]=y1/height
        box[i,1]=x1/breadth
        box[i,2]=y2/height
        box[i,3]=x2/breadth
        y=np.ndarray(shape=(1,file_data.shape[0],file_data.shape[1],3))
        y[0,:,:,:]=file_data[:,:,0:3]
   
      cropped=tf.image.crop_and_resize(image=y,boxes=box,box_ind=[0]*box.shape[0],crop_size=[image_size,image_size],method='bilinear')
      sess=tf.Session()
      with sess.as_default():
          tensor=cropped
          cropped=tensor.eval()
      n=cropped.shape[0]
      gate=0
    
      for i in range(last_index+1,n):
          images[current_index,:,:,:]=cropped[i,:,:,:]
          
          current_index=current_index+1
          if label_gate==0:
            tag_id.append(lb[i])
          else:
            tag_id.extend(lb)
          if current_index%200==0:
            print(current_index)
          if current_index>=num_pictures:
            last_index=i
            gate=1
            break
            
      if last_index!=-1 and gate==0:
        last_index=-1
      image_index=image_index+1
      del file,image,file_data,image_id,lb,bb,sess,cropped,box
      

  tag_id=np.asarray(tag_id)
 
  if last_index!=-1:
    image_index-=1
  
  return images,tag_id,image_index,last_index




In [11]:

val_image_index=2520
last_index=2
for i in range(1):
  X_val,tag_id,val_image_index,last_index=get_next_val_batch(val_image_index,379,last_index)
  print('val_image_index',val_image_index)
  print('last_index',last_index)
  prob=mymodel.predict(X_val)
  df1=pd.DataFrame(data=prob, index=None,dtype='float32', columns=['black', 'blue', 'green', 'other', 'red', 'silver/grey', 'white','yellow'])
  df2=pd.DataFrame(data=tag_id,index=None,dtype='int32',columns=['tag_id'])
  df=pd.concat([df2,df1],axis=1)
  df.to_csv("./gdrive/My Drive/mafat/color/color_prob6.csv")
  del X_val,tag_id,prob,df1,df2,df
  





200
val_image_index 2541
last_index 19


In [0]:
df1 = pd.read_csv("./gdrive/My Drive/mafat/color/color_prob1.csv")
df2 = pd.read_csv("./gdrive/My Drive/mafat/color/color_prob2.csv")
df3 = pd.read_csv("./gdrive/My Drive/mafat/color/color_prob3.csv")
df4 = pd.read_csv("./gdrive/My Drive/mafat/color/color_prob4.csv")
df5 = pd.read_csv("./gdrive/My Drive/mafat/color/color_prob5.csv")
df6 = pd.read_csv("./gdrive/My Drive/mafat/color/color_prob6.csv")

df_array=[df1,df2,df3,df4,df5,df6]
df=pd.concat(df_array,axis=0)
df.to_csv("./gdrive/My Drive/mafat/color/color_prob.csv")

In [18]:
os.listdir("./gdrive/My Drive/mafat/color")

['Predict Mafat-color.ipynb',
 'Mafat-color.ipynb',
 'MyModel_color',
 'model_color',
 'color_prob1.csv',
 'color_prob2.csv',
 'color_prob3.csv',
 'color_prob4.csv',
 'color_prob5.csv',
 'color_prob6.csv',
 'color_prob.csv']

In [0]:
df = pd.read_csv("./gdrive/My Drive/mafat/color/color_prob.csv")
df.to_csv("./gdrive/My Drive/mafat/color_prob.csv")